Задание
Основные цели этого задания:

Попрактиковаться в борьбе с дисбалансом классов

Научиться заполнять пропуски в данных

Научиться использовать категориальные признаки.

Задача: по различным признакам, связанным с заявкой на грант, предсказать, будет ли заявка принята. 

План решения:

1. Загрузите данные из csv файла. Ознакомьтесь с ними, проверьте наличие пропусков, узнайте типы признаков.

In [76]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn. model_selection import cross_val_score

In [77]:
df = pd.read_csv('grant_data_imb.csv')
df.head().T

,0,1,2,3,4
Grant.Status,0,0,0,0,0
Sponsor.Code,97A,36D,317A,62B,1A
Grant.Category.Code,30B,10A,30D,10B,10A
Contract.Value.Band...see.note.A,A,G,NaN,B,NaN
RFCD.Code.1,321024.0,300201.0,321013.0,321103.0,270603.0
RFCD.Percentage.1,50.0,100.0,100.0,30.0,60.0
RFCD.Code.2,321013.0,0.0,0.0,321105.0,321205.0
RFCD.Percentage.2,30.0,0.0,0.0,40.0,30.0
RFCD.Code.3,291502.0,0.0,0.0,321204.0,320603.0
RFCD.Percentage.3,20.0,0.0,0.0,30.0,10.0


1.1 Ознакомьтесь с ними, проверьте наличие пропусков, узнайте типы признаков.

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 39 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Grant.Status                            4113 non-null   int64  
 1   Sponsor.Code                            3856 non-null   object 
 2   Grant.Category.Code                     3856 non-null   object 
 3   Contract.Value.Band...see.note.A        1953 non-null   object 
 4   RFCD.Code.1                             3853 non-null   float64
 5   RFCD.Percentage.1                       3853 non-null   float64
 6   RFCD.Code.2                             3853 non-null   float64
 7   RFCD.Percentage.2                       3853 non-null   float64
 8   RFCD.Code.3                             3853 non-null   float64
 9   RFCD.Percentage.3                       3853 non-null   float64
 10  RFCD.Code.4                             3853 non-null   floa

2. Подготовьте данные к обучению моделей:

Отделите целевую переменную Grant.Status;

Заполните пропуски

- в количественных признаках заполните пропуски средними значениями и нулями (у каждой фичи будет по два варианта),

- в категориальных признаках выведите значения, которые они принимают, и найдите признак, где пропуски можно заполнить осмысленным значением. В остальных признаках заполните пропуски значением по умолчанию;

Преобразуйте категориальные признаки в количественные с помощью прямого кодирования;

Разделите данные на обучающую и тестовую части;

Используйте масштабирование для получения признаков одинакового масштаба.

In [79]:
# Отделение признаков и целевой переменной
X = df.drop(columns=['Grant.Status'], errors='ignore')
y = df.get('Grant.Status')

In [80]:
# обнуление пропусков
num_columns = ['RFCD.Code.1',
               'RFCD.Percentage.1',
               'RFCD.Code.2',
               'RFCD.Percentage.2',
               'RFCD.Code.3',
               'RFCD.Percentage.3',
               'RFCD.Code.4',
               'RFCD.Percentage.4',
               'RFCD.Code.5',
               'RFCD.Percentage.5',
               'SEO.Code.1',
               'SEO.Percentage.1',
               'SEO.Code.2',
               'SEO.Percentage.2',
               'SEO.Code.3',
               'SEO.Percentage.3',
               'SEO.Code.4',
               'SEO.Percentage.4',
               'SEO.Code.5',
               'SEO.Percentage.5',
               'Person.ID.1',
               'Dept.No..1',
               'Faculty.No..1',
               'Number.of.Successful.Grant.1',
               'Number.of.Unsuccessful.Grant.1',
               'A..1',
               'A.1',
               'B.1',
               'C.1']

for col in num_columns:
    X.fillna({col: 0}, inplace=True)

# замена пропусков на среднее значение
X.fillna({'Year.of.Birth.1': int(df['Year.of.Birth.1'].mean())}, inplace=True)

In [81]:
X[X['Home.Language.1'].isna()]['Country.of.Birth.1'].unique()

array(['Eastern Europe', 'Australia', 'Great Britain', 'North America',
       'Western Europe', 'Asia Pacific', nan, 'New Zealand',
       'Middle East and Africa', 'South Africa', 'The Americas'],
      dtype=object)

In [82]:
# замена пропусков на 'Unknown'
cat_columns = ['Sponsor.Code', 'Grant.Category.Code', 'Contract.Value.Band...see.note.A',
                  'Role.1', 'Country.of.Birth.1', 'No..of.Years.in.Uni.at.Time.of.Grant.1']

for col in cat_columns:
    X.fillna({col: 'Unknown'}, inplace=True)

# замена пропусков на значения из словаря
countries_dict = {
    'Australia':'English',
    'Great Britain':'English',
    'North America':'English',
    'New Zealand':'English',
    'Middle East and Africa':'English',
    'The Americas':'English',
    'Unknown':'Other',
    'South Africa':'Other',
    'Eastern Europe':'Other',
    'Western Europe':'Other',
    'Asia Pacific':'Other',
    'The Americas':'Other'
}

X['Home.Language.1'] = X.apply(lambda f: f['Home.Language.1'] if pd.notna(f['Home.Language.1'])\
                                 else countries_dict[f['Country.of.Birth.1']], axis=1)

# обнуление пропусков + цифровое кодирование
X['With.PHD.1'] = X['With.PHD.1'].apply(lambda x: 1 if x == 'Yes ' else 0)

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Sponsor.Code                            4113 non-null   object 
 1   Grant.Category.Code                     4113 non-null   object 
 2   Contract.Value.Band...see.note.A        4113 non-null   object 
 3   RFCD.Code.1                             4113 non-null   float64
 4   RFCD.Percentage.1                       4113 non-null   float64
 5   RFCD.Code.2                             4113 non-null   float64
 6   RFCD.Percentage.2                       4113 non-null   float64
 7   RFCD.Code.3                             4113 non-null   float64
 8   RFCD.Percentage.3                       4113 non-null   float64
 9   RFCD.Code.4                             4113 non-null   float64
 10  RFCD.Percentage.4                       4113 non-null   floa

In [83]:
# преобразование количественных столбцов в категориальные
num_to_object_columns = [ 
    'RFCD.Code.1', 
    'RFCD.Code.2', 
    'RFCD.Code.3', 
    'RFCD.Code.4', 
    'RFCD.Code.5', 
    'SEO.Code.1', 
    'SEO.Code.2', 
    'SEO.Code.3', 
    'SEO.Code.4', 
    'SEO.Code.5'
]

for col in num_to_object_columns:
    X[col] = X[col].astype(object)

# прямое кодирование
encoded_X = pd.get_dummies(X, drop_first=True)

for i in range(1,6):
    for col in encoded_X.columns:
        if col.startswith(f'RFCD.Code.{i}'):
            encoded_X[col] = encoded_X[col] * encoded_X[f'RFCD.Percentage.{i}']

        elif col.startswith('SEO.Code.{i}'):
            encoded_X[col] = encoded_X[col] * encoded_X[f'SEO.Percentage.{i}']

    encoded_X.drop(columns=[f'RFCD.Percentage.{i}'], inplace=True)
    encoded_X.drop(columns=[f'SEO.Percentage.{i}'], inplace=True)

encoded_X.head().T

,0,1,2,3,4
Person.ID.1,67037.0,12917.0,28607.0,144632.0,1797.0
Year.of.Birth.1,1950.0,1955.0,1965.0,1960.0,1960.0
Dept.No..1,2563.0,1038.0,2763.0,2848.0,2678.0
Faculty.No..1,25.0,1.0,25.0,25.0,25.0
With.PHD.1,1,0,1,0,1
...,...,...,...,...,...
No..of.Years.in.Uni.at.Time.of.Grant.1_>5 to 10,False,False,True,False,True
No..of.Years.in.Uni.at.Time.of.Grant.1_>=0 to 5,False,False,False,False,False
No..of.Years.in.Uni.at.Time.of.Grant.1_Less than 0,False,True,False,True,False
No..of.Years.in.Uni.at.Time.of.Grant.1_Unknown,False,False,False,False,False


In [84]:
encoded_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Columns: 2877 entries, Person.ID.1 to No..of.Years.in.Uni.at.Time.of.Grant.1_more than 15
dtypes: bool(1261), float64(1615), int64(1)
memory usage: 55.7 MB


In [85]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(encoded_X, y, test_size=0.2, random_state=42)

In [86]:
# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

X_test_scaled.head().T

,0,1,2,3,4
Person.ID.1,-0.076273,-0.187023,-0.110751,-0.316972,-0.308079
Year.of.Birth.1,0.806280,0.806280,0.256164,-0.844070,-0.844070
Dept.No..1,0.895699,-0.011273,0.464038,1.050903,0.076029
Faculty.No..1,0.770995,-0.173432,0.298781,0.770995,-0.173432
With.PHD.1,0.875738,0.875738,0.875738,0.875738,0.875738
...,...,...,...,...,...
No..of.Years.in.Uni.at.Time.of.Grant.1_>5 to 10,-0.458747,-0.458747,-0.458747,-0.458747,-0.458747
No..of.Years.in.Uni.at.Time.of.Grant.1_>=0 to 5,1.435432,1.435432,1.435432,-0.696654,-0.696654
No..of.Years.in.Uni.at.Time.of.Grant.1_Less than 0,-0.372033,-0.372033,-0.372033,-0.372033,-0.372033
No..of.Years.in.Uni.at.Time.of.Grant.1_Unknown,-0.476127,-0.476127,-0.476127,-0.476127,-0.476127


3. Изучите распределение по целевой переменной, чтобы выяснить, сбалансированы ли классы. Если классы не сбалансированы, используйте в работе хотя бы один из изученных методов борьбы с дисбалансом классов

In [87]:
y.value_counts()

Grant.Status
0    3259
1     854
Name: count, dtype: int64

In [88]:
y_train.value_counts()

Grant.Status
0    2614
1     676
Name: count, dtype: int64

In [89]:
y_test.value_counts()

Grant.Status
0    645
1    178
Name: count, dtype: int64

In [90]:
def upsample(features, target, repeat=10):
    # разделяем объекты разных классов и информацию о них по разным переменным
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    # дублируем записи объектов более редкого класса
    if len(target_ones) > len(target_zeros):
        repeat = round(len(target_ones) / len(target_zeros))
        features_upsampled = pd.concat([features_ones] + [features_zeros] * repeat)
        target_upsampled = pd.concat([target_ones] + [target_zeros] * repeat)
    else:
        repeat = round(len(target_zeros) / len(target_ones))
        features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
        target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    # перемешиваем объекты
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=23)
    
    return features_upsampled, target_upsampled

In [91]:
X_train_upsampled, y_train_upsampled = upsample(X_train_scaled, y_train.reset_index(drop=True))
X_test_upsampled, y_test_upsampled = upsample(X_test_scaled, y_test.reset_index(drop=True))

In [92]:
y_train_upsampled.value_counts()

Grant.Status
1    2704
0    2614
Name: count, dtype: int64

In [93]:
y_test_upsampled.value_counts()

Grant.Status
1    712
0    645
Name: count, dtype: int64

Обучите модели и выберите лучшую:

Обучите модель логистической регрессии, используя кросс-валидацию. Оцените ее качество с помощью метрики rocauc. Выведите топ-10 признаков по важности, согласно обученной модели;

In [94]:
model_log_reg_cv = LogisticRegressionCV(solver='liblinear', random_state=12, class_weight='balanced', cv=5)
model_log_reg_cv.fit(X_train_upsampled, y_train_upsampled)
roc_auc_score(y_test_upsampled, model_log_reg_cv.predict_proba(X_test_upsampled)[:, 1])

np.float64(0.8123726156258166)

In [99]:
top_10_log_reg_features = pd.DataFrame(
    zip(list(encoded_X.columns), list(abs(model_log_reg_cv.coef_[0]))), 
    columns=['feature', 'importance']
).sort_values(by=['importance'], ascending=False).head(10)
top_10_log_reg_features

,feature,importance
265,Contract.Value.Band...see.note.A_Unknown,1.314719
6,Number.of.Unsuccessful.Grant.1,1.210307
5,Number.of.Successful.Grant.1,0.834888
249,Grant.Category.Code_Unknown,0.416300
236,Sponsor.Code_Unknown,0.416300
203,Sponsor.Code_59C,0.396323
213,Sponsor.Code_6B,0.385563
258,Contract.Value.Band...see.note.A_J,0.385326
117,Sponsor.Code_24D,0.344285
460,RFCD.Code.1_291504.0,0.339199


In [96]:
# Обучение модели случайного леса с GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train_upsampled, y_train_upsampled)

# Лучшие параметры
best_rf = grid_search.best_estimator_
print(f'Лучшие параметры случайного леса: {grid_search.best_params_}')

# Оценка качества выбранной модели
best_rf_roc_auc = roc_auc_score(y_test_upsampled, best_rf.predict_proba(X_test_upsampled)[:, 1], multi_class='ovr')
print(f'ROC AUC для случайного леса: {best_rf_roc_auc:.4f}')

# Топ-10 признаков по важности
rf_feature_importance = pd.Series(best_rf.feature_importances_, index=X_train_upsampled.columns)
top_10_rf_features = rf_feature_importance.nlargest(10)
print("Топ-10 признаков по важности (случайный лес):")
print(top_10_rf_features)

Лучшие параметры случайного леса: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
ROC AUC для случайного леса: 0.9389
Топ-10 признаков по важности (случайный лес):
Contract.Value.Band...see.note.A_Unknown    0.109816
Number.of.Unsuccessful.Grant.1              0.081970
Number.of.Successful.Grant.1                0.051423
Person.ID.1                                 0.027937
Dept.No..1                                  0.023456
Grant.Category.Code_Unknown                 0.021456
Sponsor.Code_Unknown                        0.020941
B.1                                         0.020870
A.1                                         0.020817
A..1                                        0.019253
dtype: float64


5. Напишите вывод, в котором будет содержаться информация о том, какие признаки важны согласно обеим моделям, и какая модель оказалась наилучшей для решения поставленной задачи.

### Вывод:

Модель случайного леса лучше, чем логистическая регрессия, т.к. roc auc выше: 0.9389 против 0.8123726156258166.

Согласно обеим моделям важны следующие признаки:
- 'Contract.Value.Band...see.note.A_Unknown',
- 'Number.of.Successful.Grant.1',
- 'Number.of.Unsuccessful.Grant.1',
- 'Grant.Category.Code_Unknown',
- 'Sponsor.Code_Unknown'